In [ ]:
'''
for i in range(8,9):
    if i<10:
        url="https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/AQUA_MODIS.2019070"+str(i)+".L3m.DAY.SST.sst.4km.nc"
    else:
        url="https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/AQUA_MODIS.201907"+str(i)+".L3m.DAY.SST.sst.4km.nc"
    file = pooch.retrieve(url,processor=None,known_hash=None)
'''    
        

In [ ]:
'''
def file_load(i):
    dir='/Users/skerhalkar/Documents/Python Crap/Python-2/requested_files/AQUA_MODIS.201907'
    
    if i<10:
            ds=xr.open_dataset(dir+'0'+str(i)+'.L3m.DAY.SST.x_sst.nc')
    else:
            ds=xr.open_dataset(dir+str(i)+'.L3m.DAY.SST.x_sst.nc')
    
    return(ds)
'''

In [ ]:
ds = []
for fname in glob(path):
    dsi = xr.open_dataset(fname)
    dsi = dsi.assign_coords(time=parser.parse(fname.split('/')[-1].split('.')[1])).expand_dims('time')
    # dsi = dsi.sel(lon=slice(),lat=slice())
    ds.append(dsi)
ds = xr.concat(ds,dim='time')

In [ ]:
ds = xr.open_mfdataset()

In [ ]:
ds.sst.mean('time').plot(robust=True)

In [ ]:
dsi = xr.open_dataset(fname)

In [ ]:
dsi.assign_coords(time=parser.parse(fname.split('/')[-1].split('.')[1])).expand_dims('time')

In [ ]:
parser.parse(fname.split('/')[-1].split('.')[1])

In [ ]:
ds=file_load(15)
ds`

In [2]:
'''
cs = (
    ds.sla.squeeze()
    .sel(latitude=slice(15,20))
    .plot.contourf(ax=ax,levels=levels,extend='both', transform=ccrs.PlateCarree())
)
'''